<a href="https://colab.research.google.com/github/bishair/Pirna/blob/main/Print.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from google.colab import files
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from math import sqrt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [4]:
uploaded_file = files.upload()

Saving riverL.xlsx to riverL.xlsx


In [5]:
# Read the groundwater level data
gw_data = pd.read_excel('groundwater.xlsx')
gw_data['Date'] = pd.to_datetime(gw_data['Date'], format='%d/%m/%Y %H:%M')

In [6]:
print(gw_data)

                     Date      G10
0     2015-01-30 12:18:00  110.722
1     2015-01-30 13:18:00  110.720
2     2015-01-30 14:18:00  110.718
3     2015-01-30 15:18:00  110.715
4     2015-01-30 16:18:00  110.714
...                   ...      ...
17533 2017-01-30 10:19:00  109.456
17534 2017-01-30 11:19:00  109.455
17535 2017-01-30 12:19:00  109.456
17536 2017-01-30 13:19:00  109.457
17537 2017-01-30 14:19:00  109.459

[17538 rows x 2 columns]


In [7]:
 # Read the river water level data
river_data = pd.read_excel('riverL.xlsx')
river_data['Date'] = pd.to_datetime(river_data['Date'], format='%d/%m/%Y %H:%M')


In [8]:
print(river_data)

                     Date  River
0     2014-12-31 23:00:00    192
1     2015-01-02 08:00:00    206
2     2015-01-02 20:00:00    207
3     2015-01-04 08:00:00    211
4     2015-01-04 14:00:00    207
...                   ...    ...
23772 2017-12-31 19:00:00    260
23773 2017-12-31 20:00:00    260
23774 2017-12-31 21:00:00    260
23775 2017-12-31 22:00:00    260
23776 2017-12-31 23:00:00    260

[23777 rows x 2 columns]


In [9]:
 # Merge the two datasets on the 'Date' column
merged_data = pd.merge_asof(gw_data.sort_values('Date'), river_data.sort_values('Date'), on='Date', direction='nearest')

In [10]:
print(merged_data)

                     Date      G10  River
0     2015-01-30 12:18:00  110.722    261
1     2015-01-30 13:18:00  110.720    260
2     2015-01-30 14:18:00  110.718    260
3     2015-01-30 15:18:00  110.715    260
4     2015-01-30 16:18:00  110.714    260
...                   ...      ...    ...
17533 2017-01-30 10:19:00  109.456    136
17534 2017-01-30 11:19:00  109.455    138
17535 2017-01-30 12:19:00  109.456    141
17536 2017-01-30 13:19:00  109.457    144
17537 2017-01-30 14:19:00  109.459    146

[17538 rows x 3 columns]


In [11]:
 # Set 'Date' as the index
merged_data.set_index('Date', inplace=True)

In [12]:

print(merged_data)

                         G10  River
Date                               
2015-01-30 12:18:00  110.722    261
2015-01-30 13:18:00  110.720    260
2015-01-30 14:18:00  110.718    260
2015-01-30 15:18:00  110.715    260
2015-01-30 16:18:00  110.714    260
...                      ...    ...
2017-01-30 10:19:00  109.456    136
2017-01-30 11:19:00  109.455    138
2017-01-30 12:19:00  109.456    141
2017-01-30 13:19:00  109.457    144
2017-01-30 14:19:00  109.459    146

[17538 rows x 2 columns]


In [55]:
# Create a new date range that starts from the first timestamp in merged_data
start_date = merged_data.index.min()
end_date = merged_data.index.max()
new_date_range = pd.date_range(start=start_date, end=end_date, freq='H')

# Reindex and resample merged_data with the new date range
resampled_data = merged_data.reindex(new_date_range).ffill().bfill()
# Resample the data to hourly frequency and forward fill missing values
#resampled_data = merged_data.resample('H').ffill()



In [56]:
print(resampled_data)

                         G10  River
2015-01-30 12:18:00  110.722  261.0
2015-01-30 13:18:00  110.720  260.0
2015-01-30 14:18:00  110.718  260.0
2015-01-30 15:18:00  110.715  260.0
2015-01-30 16:18:00  110.714  260.0
...                      ...    ...
2017-01-30 10:18:00  110.019  199.0
2017-01-30 11:18:00  110.019  199.0
2017-01-30 12:18:00  110.019  199.0
2017-01-30 13:18:00  110.019  199.0
2017-01-30 14:18:00  110.019  199.0

[17547 rows x 2 columns]


# Preprocess DATA for LSTM model input

In [57]:
# Handling missing values
data = resampled_data.fillna(method='ffill')  # Forward fill
print(data)

                         G10  River
2015-01-30 12:18:00  110.722  261.0
2015-01-30 13:18:00  110.720  260.0
2015-01-30 14:18:00  110.718  260.0
2015-01-30 15:18:00  110.715  260.0
2015-01-30 16:18:00  110.714  260.0
...                      ...    ...
2017-01-30 10:18:00  110.019  199.0
2017-01-30 11:18:00  110.019  199.0
2017-01-30 12:18:00  110.019  199.0
2017-01-30 13:18:00  110.019  199.0
2017-01-30 14:18:00  110.019  199.0

[17547 rows x 2 columns]


In [58]:
 # Normalizing the data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

In [59]:
print(data_scaled)
print(data_scaled.shape)

[[1.         0.95238095]
 [0.99738903 0.94047619]
 [0.99477807 0.94047619]
 ...
 [0.08224543 0.21428571]
 [0.08224543 0.21428571]
 [0.08224543 0.21428571]]
(17547, 2)


In [60]:
# Define window size
n_steps = 5

X, y = [], []
for i in range(n_steps, len(data_scaled)):
  X.append(data_scaled[i-n_steps:i, :])
  y.append(data_scaled[i, 0])
X, y = np.array(X), np.array(y)



In [61]:
print (X)
print(X.shape)

[[[1.         0.95238095]
  [0.99738903 0.94047619]
  [0.99477807 0.94047619]
  [0.99086162 0.94047619]
  [0.98955614 0.94047619]]

 [[0.99738903 0.94047619]
  [0.99477807 0.94047619]
  [0.99086162 0.94047619]
  [0.98955614 0.94047619]
  [0.98694517 0.94047619]]

 [[0.99477807 0.94047619]
  [0.99086162 0.94047619]
  [0.98955614 0.94047619]
  [0.98694517 0.94047619]
  [0.9843342  0.95238095]]

 ...

 [[0.08224543 0.21428571]
  [0.08224543 0.21428571]
  [0.08224543 0.21428571]
  [0.08224543 0.21428571]
  [0.08224543 0.21428571]]

 [[0.08224543 0.21428571]
  [0.08224543 0.21428571]
  [0.08224543 0.21428571]
  [0.08224543 0.21428571]
  [0.08224543 0.21428571]]

 [[0.08224543 0.21428571]
  [0.08224543 0.21428571]
  [0.08224543 0.21428571]
  [0.08224543 0.21428571]
  [0.08224543 0.21428571]]]
(17542, 5, 2)


In [62]:
print(y)
print(y.shape)

[0.98694517 0.9843342  0.98302872 ... 0.08224543 0.08224543 0.08224543]
(17542,)


In [63]:
# Check if there are any NaN values in X
nan_in_X = np.isnan(X).any()
nan_count_X = np.isnan(X).sum()

# Check if there are any NaN values in y
nan_in_y = np.isnan(y).any()
nan_count_y = np.isnan(y).sum()

print(f"NaN values in X? {nan_in_X}")
print(f"Total number of NaN values in X: {nan_count_X}")

print(f"NaN values in y? {nan_in_y}")
print(f"Total number of NaN values in y: {nan_count_y}")

NaN values in X? False
Total number of NaN values in X: 0
NaN values in y? False
Total number of NaN values in y: 0


In [64]:
train_size = int(len(X) * 0.7)
test_size = len(X) - train_size
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(train_size)
print(test_size)
print(train_size+test_size)
print(X_train.shape, 'X-train shape')
print(X_test.shape, 'X-test shape')
print(y_train.shape, 'y-train shape')
print(X_test.shape, 'y-test shape')

12279
5263
17542
(12279, 5, 2) X-train shape
(5263, 5, 2) X-test shape
(12279,) y-train shape
(5263, 5, 2) y-test shape


In [65]:
input_shape = X_train.shape[1], X_train.shape[2]
model = Sequential([
        LSTM(units=50, return_sequences=True, input_shape=input_shape),
        LSTM(units=50),
        Dense(1)
    ])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=40, batch_size=64)

Epoch 1/40
192/192 [==============================] - 4s 5ms/step - loss: 8.6483e-04
Epoch 2/40
192/192 [==============================] - 1s 6ms/step - loss: 2.2252e-06
Epoch 3/40
192/192 [==============================] - 1s 5ms/step - loss: 1.9560e-06
Epoch 4/40
192/192 [==============================] - 1s 5ms/step - loss: 1.8652e-06
Epoch 5/40
192/192 [==============================] - 1s 5ms/step - loss: 1.9416e-06
Epoch 6/40
192/192 [==============================] - 1s 5ms/step - loss: 1.8040e-06
Epoch 7/40
192/192 [==============================] - 1s 6ms/step - loss: 2.6673e-06
Epoch 8/40
192/192 [==============================] - 1s 6ms/step - loss: 3.6685e-06
Epoch 9/40
192/192 [==============================] - 1s 5ms/step - loss: 2.7044e-06
Epoch 10/40
192/192 [==============================] - 1s 6ms/step - loss: 3.1347e-06
Epoch 11/40
192/192 [==============================] - 1s 8ms/step - loss: 2.3880e-06
Epoch 12/40
192/192 [==============================] - 1s 5ms/s

In [66]:
print(model)
trained_model = model
print(trained_model)